# Keyword recognition with PyTorch

In [1]:
import os
import torch
import torchaudio
import matplotlib.pyplot as plt
import numpy as np
from torch.utils.data import Dataset, DataLoader

import torchaudio.transforms as T
import torchaudio.functional as F

Load the Mini Speech dataset

In [ ]:
def set_set_to_16000(tensor):
    if tensor.size()[0] < 16000:
        tensor = tensor[:160000, ...]

In [23]:
class MyAudioDataset(Dataset):
    def __init__(self, root_dir, sample_rate=16000, transform=None):
        self.root_dir = root_dir
        self.sample_rate = sample_rate
        self.transform = transform

        # Collect all file paths and labels
        self.file_paths = []
        self.labels = []
        self.label_names = sorted(os.listdir(root_dir))

        for label_idx, label_name in enumerate(self.label_names):
            label_folder = os.path.join(root_dir, label_name)
            for file_name in os.listdir(label_folder):
                if file_name.endswith(".wav"):
                    self.file_paths.append(os.path.join(label_folder, file_name))
                    self.labels.append(label_idx)


    def __len__(self):
        return len(self.file_paths)

    def __getitem__(self, idx):
        file_path = self.file_paths[idx]
        label = self.labels[idx]

        waveform, sr = torchaudio.load(file_path)

        # Resample if needed
        if sr != self.sample_rate:
            waveform =  F.resample(
            waveform,
            sr,
            16000,
            lowpass_filter_width=64,
            rolloff=0.9475937167399596,
            resampling_method="sinc_interp_kaiser",
            beta=14.769656459379492,
        )

        # Convert to mono
        if waveform.shape[0] > 1:
            waveform = torch.mean(waveform, dim=0, keepdim=True)

        # Apply transforms (e.g. MelSpectrogram)
        if self.transform:
            features = self.transform(waveform)
        else:
            features = waveform

        # Pad shorter clips, trim longer clips
        max_len = self.sample_rate  # 1 second = 16000 samples
        if waveform.shape[1] < max_len:
            waveform = torch.nn.functional.pad(waveform, (0, max_len - waveform.shape[1]))
        else:
            waveform = waveform[:, :max_len]
        return features, label

In [30]:
transform = T.MelSpectrogram(sample_rate=16000, n_mels=64)

train_dataset = MyAudioDataset("data2", sample_rate=16000, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)

## CNN Architecture

In [31]:
import torch.nn as nn
import torch.nn.functional as F

In [32]:
class KeywordCNN(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(16)
        
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(32)
        
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.bn3 = nn.BatchNorm2d(64)
        
        self.pool = nn.MaxPool2d(2, 2)
        self.dropout = nn.Dropout(0.3)
        
        # Global average pooling
        self.fc = nn.Linear(64, num_classes)

    def forward(self, x):
        # x: [batch, 1, n_mels, time]
        x = self.pool(F.relu(self.bn1(self.conv1(x))))
        x = self.pool(F.relu(self.bn2(self.conv2(x))))
        x = self.pool(F.relu(self.bn3(self.conv3(x))))
        
        # Global average pooling
        x = x.mean(dim=[2, 3])  # [batch, channels]
        x = self.dropout(x)
        x = self.fc(x)
        return x

In [33]:
model = KeywordCNN(num_classes=len(train_dataset.label_names))

Minimal training loop

In [35]:
import torch.optim as optim

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

for epoch in range(100):
    model.train()
    running_loss = 0.0
    for features, labels in train_loader:
        features, labels = features.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(features)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader):.4f}")

Epoch 1, Loss: 1.5121
Epoch 2, Loss: 1.4762
Epoch 3, Loss: 1.4347
Epoch 4, Loss: 1.4367
Epoch 5, Loss: 1.3767
Epoch 6, Loss: 1.3973
Epoch 7, Loss: 1.3157
Epoch 8, Loss: 1.3332
Epoch 9, Loss: 1.3420
Epoch 10, Loss: 1.2718
Epoch 11, Loss: 1.1757
Epoch 12, Loss: 1.1853
Epoch 13, Loss: 1.1572
Epoch 14, Loss: 1.1414
Epoch 15, Loss: 1.1037
Epoch 16, Loss: 1.1328
Epoch 17, Loss: 1.0616
Epoch 18, Loss: 1.0081
Epoch 19, Loss: 1.0036
Epoch 20, Loss: 0.9243
Epoch 21, Loss: 0.9305
Epoch 22, Loss: 0.9930
Epoch 23, Loss: 0.9642
Epoch 24, Loss: 0.8379
Epoch 25, Loss: 0.8548
Epoch 26, Loss: 0.8016
Epoch 27, Loss: 0.8089
Epoch 28, Loss: 0.7047
Epoch 29, Loss: 0.8391
Epoch 30, Loss: 0.6684
Epoch 31, Loss: 0.6936
Epoch 32, Loss: 0.6587
Epoch 33, Loss: 0.6647
Epoch 34, Loss: 0.6442
Epoch 35, Loss: 0.5361
Epoch 36, Loss: 0.5299
Epoch 37, Loss: 0.6270
Epoch 38, Loss: 0.5524
Epoch 39, Loss: 0.5485
Epoch 40, Loss: 0.5475
Epoch 41, Loss: 0.4583
Epoch 42, Loss: 0.5065
Epoch 43, Loss: 0.4385
Epoch 44, Loss: 0.43

In [39]:
# Save the entire model to PATH
PATH = 'model/kws_model.pth'
torch.save(model.state_dict(), PATH)



Testing on training data

In [40]:
def predict_file(model, filepath, transform, sample_rate, label_names, device):
    # Load audio
    waveform, sr = torchaudio.load(filepath)
    
    # Resample if needed
    if sr != sample_rate:
        waveform = torchaudio.transforms.Resample(orig_freq=sr, new_freq=sample_rate)(waveform)
    
    # Convert to mono
    if waveform.shape[0] > 1:
        waveform = torch.mean(waveform, dim=0, keepdim=True)
    
    # Pad or trim to 1 second (for 16kHz = 16000 samples)
    max_len = sample_rate
    if waveform.shape[1] < max_len:
        waveform = F.pad(waveform, (0, max_len - waveform.shape[1]))
    else:
        waveform = waveform[:, :max_len]
    
    # Apply transform (e.g. MelSpectrogram)
    features = transform(waveform).unsqueeze(0).to(device)  # [1, 1, n_mels, time]
    
    # Run model
    model.eval()
    with torch.no_grad():
        outputs = model(features)
        pred_idx = torch.argmax(outputs, dim=1).item()
    
    pred_label = label_names[pred_idx]
    print(f"🎙️ Predicted keyword: {pred_label}")
    return pred_label

In [56]:
# Example configuration
sample_rate = 16000  # or whatever your model was trained on

# Assuming you already defined your transform, e.g.:
# transform = torchaudio.transforms.MelSpectrogram(sample_rate=16000, n_mels=64)

# And you have a trained model + label names
file_path = "data2/ok/ok_hasan.wav"

predict_file(model, file_path, transform, sample_rate, train_dataset.label_names, device)

🎙️ Predicted keyword: fernando


'fernando'